In [1]:
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# quick EDA analyses
#import pandas_profiling as pd_profile

# Set today's date
import datetime as dt
from datetime import datetime, timedelta
import calendar
today = dt.datetime.today().strftime("%Y-%m-%d")

#date of data extraction
date_of_classification = dt.datetime(2020,2,21,9,0,0)  # replace date with year,month,day,hour,minute,sec

# Set current time
import time 
start_time = time.time()


# No warnings about setting value on copy of slice
pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", 150)
pd.set_option("display.max_rows", 10000)
pd.set_option("display.float_format", lambda x: '%.2f' % x)


# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings('ignore')

#in-house tools for columns renaming, dates transformation and checking missing values
from sfl_tools import *
# computing Weight of Evidence (WOE) and Information value(WOE)
from WOE import *

# for splitting flatfile into training and test set
from sklearn.model_selection import train_test_split

In [2]:
#data dictionaries
data_directory = "../data/"
data_output_directory = data_directory + "notebook_outputs/" 
data_flatfile_directory = data_directory + "flatfile/"
metropol_data_output_directory =  data_output_directory + "metropol_summary_features/"

In [3]:
interna_data = pd.read_csv(data_flatfile_directory + "flatfile_survey_only.csv")
internal_data_pre_acceptance = pd.read_csv(data_output_directory + "survey_features_flatfile_pre_acceptance.csv")

In [4]:
interna_data.columns

Index(['Unnamed: 0', 'farmer_age.group', 'farm_size_stayed_same',
       'yield_kg.group', 'other_non-farm_income.binary', 'education_level',
       'schooling_years.group', 'dependants.group', 'other_crops.binary',
       'cooperative_member', 'agric_loans_type.inputs',
       'agric_loans_type.mechanisation', 'loans_type.inputs_mech',
       'protection_product.zetanil', 'protection_product.redomil',
       'status_by_margins_60', 'status_by_margins_30', 'order_id',
       'order_value', 'order_value.group', 'total_arrears', 'margin',
       'age_of_loan'],
      dtype='object')

In [24]:
#interna_data.dtypes

In [23]:
#internal_data_pre_acceptance.columns

In [7]:
non_modelling_features = ['Unnamed: 0','status_by_margins_60','order_id','total_arrears','margin','age_of_loan']
internal_data = interna_data.drop(non_modelling_features, axis = 1)

In [8]:
non_modelling_features = ['Unnamed: 0','status_by_margins_60','total_arrears','margin','age_of_loan',
                          'total_arrears', 'margin','county.category', 'ward.category', 'agent_name.category',
                          'loan_cohort.category','agent_name', 'county', 'ward', 'loan_cohort']
pre_acceptance_df = internal_data_pre_acceptance.drop(non_modelling_features, axis = 1)

In [9]:
X = internal_data.drop(['status_by_margins_30'], axis=1)
y = internal_data['status_by_margins_30']

In [10]:
#import pandas as pd
#import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

In [11]:


# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)


In [12]:

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

In [13]:
def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [14]:
final_iv, IV = data_vars(X,y)

In [15]:
final_iv

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,farmer_age.group,25-29,25-29,255,90,0.35,165,0.65,0.12,0.13,-0.05,0.00
1,farmer_age.group,30-39,30-39,679,247,0.36,432,0.64,0.34,0.34,-0.01,0.00
2,farmer_age.group,40-49,40-49,546,207,0.38,339,0.62,0.28,0.27,0.06,0.00
3,farmer_age.group,50-59,50-59,294,104,0.35,190,0.65,0.14,0.15,-0.05,0.00
4,farmer_age.group,60+,60+,146,53,0.36,93,0.64,0.07,0.07,-0.01,0.00
5,farmer_age.group,_<25,_<25,89,33,0.37,56,0.63,0.04,0.04,0.02,0.00
6,farm_size_stayed_same,No,No,1211,431,0.36,780,0.64,0.59,0.61,-0.04,0.00
7,farm_size_stayed_same,Yes,Yes,798,303,0.38,495,0.62,0.41,0.39,0.06,0.00
8,yield_kg.group,blank,blank,68,14,0.21,54,0.79,0.02,0.04,-0.80,0.02
9,yield_kg.group,01000-03000,01000-03000,506,188,0.37,318,0.63,0.26,0.25,0.03,0.02


In [16]:
final_iv.to_csv(data_output_directory + "internal_woe_iv_values.csv")

In [17]:
#Exporting the feature list to local folder
df_feature_list_sorted = IV.sort_values('IV', ascending=False)
df_feature_list_sorted.to_csv(data_output_directory + "internal_data_IV_feature_list.csv")

In [18]:
df_feature_list_sorted.head(20)

,VAR_NAME,IV
2,cooperative_member,0.06
15,yield_kg.group,0.02
3,dependants.group,0.02
4,education_level,0.01
9,order_value.group,0.01
8,order_value,0.01
11,other_non-farm_income.binary,0.00
14,schooling_years.group,0.00
5,farm_size_stayed_same,0.00
6,farmer_age.group,0.00


In [19]:
# X = pre_acceptance_df.drop(['status_by_margins_30'], axis=1)
# y = pre_acceptance_df['status_by_margins_30']

In [20]:
# final_iv, IV = data_vars(X,y)

In [21]:
# final_iv.to_csv(data_output_directory + "pre_acceptance_woe_iv_values.csv")

In [22]:
#Exporting the feature list to local folder
# df_feature_list_sorted = IV.sort_values('IV', ascending=False)
# df_feature_list_sorted.to_csv(data_output_directory + "pre_acceptance_IV_feature_list.csv")